In [ ]:
#This program was used to perform server based location matching to identify tweets from Indian locations.This matching is 
#in addition to local enumeration based matching done earlier in filter1

In [ ]:
import pandas as pd
import numpy as np
from sys import getsizeof , maxsize 
import re
from sys import argv
from time import sleep , time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geograpy import places
import geograpy

In [ ]:
s_=r'(\W|\b)ÜT:(\W|\b)|(\W|\b)geo(\W|\b)' #locations with these, contains coordinates instead of names (although coordinates can exist witout UT,Geo in them)

In [ ]:
class cl:
    
    def __init__(self,l1,l2,l3):
        self.list_uid1=l1#user ids corrosponding to tweet that originated from India
        self.list_uid2=l2#user ids corrosponding to tweet that didn't originate from India
        self.list_uid3=l3#user ids corrosponding to tweet, location of which couldn't be found.
        #mantaining these lists is useful to prevent unnecessary server request for a tweet whose author's location has been 
        #already identified.
        self.geo=Nominatim(user_agent="my-application",timeout=20)

    def f1(self,arg):
        
        for el in self.list_uid1:
            a,b=el
            if str(arg['user_id_str'])==str(a):
                return [True,str(b)]

        
        if str(arg['user_id_str']) in self.list_uid2:
            return [False,'Foreign']
        
        if str(arg['user_id_str']) in self.list_uid3:
            return [False,'Undetect']
        
        
        if re.search(s_,str(arg['location']))!=None:#I) Using coordinates
            x=re.sub(s_,' ',arg['location'])
            x=re.sub("\'|\"", ' ' , x)#strip off ' and " (minutes and seconds symbols) as they prevent api from working properly
            
            count=1
            t_out=20
            while(True):
                try:
                    loc=self.geo.geocode(x)
                except Exception as e:
                    
                    if str(e)=='HTTP Error 500: Internal Server Error':
                        print(e," : ",x)
                        return [False,'Undetect']
                    
                    print(e)
                    print('retrying...')
                    t_out=t_out+5
                    self.geo=Nominatim(user_agent="my-application",timeout=t_out)
                    count=count+2
                    sleep(count)
                    continue
                
                break
                
            
            sleep(1)
            if loc!=None:
                if re.search(r'(\W|\b)India(\W|\b)|हिन्दुस्तान|भारत',loc.raw['display_name'],re.IGNORECASE)!=None:
                    self.list_uid1.append( [arg['user_id_str'] , loc.raw['display_name']])#if given location is desirable then save its user id for future
                    with open('file_uid1.txt','a') as file:#changes are also saved on secondary storage because Nominatim api has various limits and this 
                                                           #may lead to unexpected terminations
                            a=arg['user_id_str']
                            b=loc.raw['display_name']
                            file.write(str(a)+'!_#_!'+str(b)+'\n')

                    return [True,str(loc.raw['display_name'])]
                else:
                    self.list_uid2.append(arg['user_id_str'])#if given location is not desirable then also save its user id for future
                    with open('file_uid2.txt','a') as file:
                        el=arg['user_id_str']
                        file.write(str(el)+'\n') 
                        
                    return [False,'Foreign']
            else:
                self.list_uid3.append(arg['user_id_str'])#if given location can't be identified by Normintam then also save its user id for future
                with open('file_uid3.txt','a') as file:
                    el=arg['user_id_str']
                    file.write(str(el)+'\n')

                return [False,'Undetect']
            
        else: #II) Using complete location string
            count=1
            t_out=20
            while(True):
                try:
                    loc=self.geo.geocode(arg['location'])#send api request with complete location string
                except Exception as e:
                    if str(e)=='HTTP Error 500: Internal Server Error':
                        print(e," : ",arg['location'])
                        return [False,'Undetect']
                    
                    print(e)
                    print('retrying...')
                    t_out=t_out+5
                    self.geo=Nominatim(user_agent="my-application",timeout=t_out)
                    count=count+2
                    sleep(count)
                    continue
                
                break
            
            sleep(1)
            if loc!=None:
                print('B: ',loc.raw['display_name'])
                if re.search(r'(\W|\b)India(\W|\b)|हिन्दुस्तान|भारत',loc.raw['display_name'],re.IGNORECASE)!=None:
                    self.list_uid1.append([arg['user_id_str'] , loc.raw['display_name'] ])
                    with open('file_uid1.txt','a') as file:
                            a=arg['user_id_str']
                            b=loc.raw['display_name']
                            file.write(str(a)+'!_#_!'+str(b)+'\n')
                            
                    return [True,str(loc.raw['display_name'])]
                else:
                    self.list_uid2.append(arg['user_id_str'])
                    with open('file_uid2.txt','a') as file:
                        el=arg['user_id_str']
                        file.write(str(el)+'\n') 
                    return [False,'Foreign']
            
            else: #III) Using each token in location string seperately (this was done because in some cases, api was able to identify location
                #with a single token but not when entire location string was used)
                
                x=re.sub(r",|-|\.|:|;|\||_|\(|\)|\{|\}|\[|\]|\'|\"|!", ' ' , arg['location'])

                L=x.split()
                L.reverse()

            
                foreign=False
                for el in L:
                    count=1
                    t_out=20
                    while(True):
                        try:
                            loc=self.geo.geocode(el)#send api request with splitted location string
                        except Exception as e:
                            if str(e)=='HTTP Error 500: Internal Server Error':
                                print(e," : ",el)
                                return [False,'Undetect']
                            print(e)
                            print('retrying...')
                            t_out=t_out+5
                            self.geo=Nominatim(user_agent="my-application",timeout=t_out)
                            count=count+2
                            sleep(count)
                            continue

                        break
                    
                    sleep(1)
                    
                    if loc!=None:
                        print('C: ',loc.raw['display_name'])
                        if re.search(r'(\W|\b)India(\W|\b)|हिन्दुस्तान|भारत',loc.raw['display_name'],re.IGNORECASE)!=None:
                            self.list_uid1.append([arg['user_id_str'] , loc.raw['display_name']])
                            with open('file_uid1.txt','a') as file:
                                a=arg['user_id_str']
                                b=loc.raw['display_name']
                                file.write(str(a)+'!_#_!'+str(b)+'\n')

                            return [True,str(loc.raw['display_name'])]
                        else:
                            foreign=True
                            
                if foreign==True:
                    self.list_uid2.append(arg['user_id_str'])
                    with open('file_uid2.txt','a') as file:
                        el=arg['user_id_str']
                        file.write(str(el)+'\n') 
                        
                    return [False,'Foreign'] #if we come out of loop and also encountered a Foreign
                else:
                    self.list_uid3.append(arg['user_id_str'])
                    with open('file_uid3.txt','a') as file:
                        el=arg['user_id_str']
                        file.write(str(el)+'\n')
                    return [False,'Undetect']
                    

In [ ]:
#obj=Nominatim(user_agent="my-application")

l1=[]
l2=[]
l3=[]

with open('file_uid1.txt','r') as file:
    for el in file:
        s=str(el)
        s=re.sub('\n','',s)
        l1.append(s.split('!_#_!'))


with open('file_uid2.txt','r') as file:
    l2=file.read().splitlines()


with open('file_uid3.txt','r') as file:
    l3=file.read().splitlines()
       
obj_cl=cl(l1,l2,l3)

df=pd.read_csv('nonForeignLoc.csv',na_values={'None','none',"NONE"})

df=df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
t_start=time()

x=df['location'].isna()
df=df.loc[df.index[~x]]#only go ahead with non NaN location

y=df.apply(obj_cl.f1,axis=1)

logic=list(map( lambda arg: arg[0], y ))
logic_=list(map( lambda arg: not arg[0], y ))
z=list(map( lambda arg: arg[1], y ))

df_a=df.loc[df.index[logic]]

df_a['det_location']=[el for el in z if el!='Foreign' and el!='Undetect']


df_b=df.loc[df.index[logic_]]
df_b['det_location']=[el for el in z if el=='Foreign' or el=='Undetect']

    
df_a.to_csv('serverIndianLoc.csv',index=None,header=True)
df_b.to_csv(r'serverForeignLoc.csv',index=None,header=True)